# Applied Machine Learning homework3

Huibo Zhao hz2480
Lingjie Xu lx2222

# Perform General Data Preprocessing

In [1]:
import os
import pandas as pd
import math
import numpy as np
import random
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV

In [2]:
# Extract those files that contain our training data
path = os.getcwd()
files = os.listdir(path)
xlsx_files = [f for f in files if f[-4:] == 'xlsx' and f[0] != '~']

# Putting all the contents into dataframe for viewing and accessing
df = pd.DataFrame()
for f in xlsx_files:
    data = pd.read_excel(f, 'FEguide')
    df = df.append(data)

In [3]:
# We extract the target value
y = df['Comb Unrd Adj FE - Conventional Fuel'].as_matrix()

# Examine if the target contains any empty or nan value
# If it does, we need to drop the row without a valid target value
# Fortunately, we don't find such row
for i in y:
    if isinstance(i,float) and math.isnan(i):
        print(i)

In [4]:
dropout_features = list()


# Professor had named a few features that we cannot use, we extract the name of those features
da = np.array(df.columns)
drop_index = list()
for i in range(len(da)):
    if "FE" in da[i]:
        drop_index.append(da[i])
    elif "MPG" in da[i]:
        drop_index.append(da[i])
    elif "CO2" in da[i]:
        drop_index.append(da[i])
    elif "Smog" in da[i]:
        drop_index.append(da[i])
    elif "Guzzler" in da[i]:
        drop_index.append(da[i])
    elif "EPA" in da[i]:
        drop_index.append(da[i])
    elif "GHG" in da[i]:
        drop_index.append(da[i])

dropout_features.extend(drop_index)
df_1 = df.drop(drop_index, axis=1) ##########first drop : drop disallowed features##########
############################################################################################

# We examine each feature and count how many nan values it have
# Save results in a dictionary

column_names = df_1.columns.values
nan_dict = {}

for column_name in column_names:
    count = 0
    for i in range(0,df_1.shape[0]):
        if isinstance(df_1.iloc[i][column_name],float) and math.isnan(df_1.iloc[i][column_name]):
            count += 1
    nan_dict[column_name] = count
    
drop_2 = list()
for key in nan_dict:
    if nan_dict[key]>=2000:
        drop_2.append(key)

dropout_features.extend(drop_2)
df_2 = df_1.drop(drop_2, axis=1) ##########second drop : drop features without enough valid entry#########
##########################################################################################################

In [5]:
# After the second drop, we left with 48 features
# We found 8 features have at least 1 nan values  
# We have to handle that 
count_dict = {}
column_names = df_2.columns.values
for column_name in column_names:
    count = 0
    for i in range(0,df_2.shape[0]):
        if isinstance(df_2.iloc[i][column_name],float) and math.isnan(df_2.iloc[i][column_name]):
            count += 1
    count_dict[column_name] = count
    
        
for key in count_dict:
    if count_dict[key] > 0:
        print(str(key) + ": " + str(count_dict[key]))


Air Aspir Method: 1973
Max Ethanol % - Gasoline: 80
Descriptor - Model Type (40 Char or less): 1131
Car/Truck Category - Cash for Clunkers Bill.: 1242
Unique Label?: 295
Label Recalc?: 187
Var Valve Timing Desc: 107
$ You Spend over 5 years (increased amount spent in fuel costs over 5 years - on label) : 1080


In [6]:
dropout_features.append("Car/Truck Category - Cash for Clunkers Bill.")
df_3 = df_2.drop("Car/Truck Category - Cash for Clunkers Bill.", axis=1) #### third drop ####
####### Car/Truck features is useless after examining it ####################################

dropout_features.append("Release Date") ###### Fourth drop ##################################
df_4 = df_3.drop("Release Date", axis=1) ##### We temporarily drop the date here ############

dropout_features.append("Air Aspir Method")
dropout_features.append("Descriptor - Model Type (40 Char or less)")
df_5 = df_4.drop(["Air Aspir Method", "Descriptor - Model Type (40 Char or less)"], axis=1)

#################### We have done all dropping features by this point #######################
#################### All dropout features are stored in dropout_features ####################


In [7]:
################################ We fill out those four features manually ###################
index_ = df_5.columns
adjust_index = list()
adjust_index.append(list(index_).index("Max Ethanol % - Gasoline"))
adjust_index.append(list(index_).index("Unique Label?"))
adjust_index.append(list(index_).index("Label Recalc?"))
adjust_index.append(list(index_).index("Var Valve Timing Desc"))
adjust_index

for i in adjust_index:
    for j in range(df_5.shape[0]):
        if j != df_5.shape[0] - 1:
            if isinstance(df_5.iat[j, i], float) and math.isnan(df_5.iat[j, i]):
                if isinstance(df_5.iat[j + 1, i], float) and math.isnan(df_5.iat[j + 1, i]):
                    df_5.iat[j, i] = df_5.iat[j - 1, i]
                else:
                    m = random.randint(0,1)
                    if m == 0:
                        df_5.iat[j, i] = df_5.iat[j - 1, i]
                    else:
                        df_5.iat[j, i] = df_5.iat[j + 1, i]
        else:
            if isinstance(df_5.iat[j, i],float) and math.isnan(df_5.iat[j, i]):
                df_5.iat[j, i] = df_5.iat[j - 1, i]
##############################################################################################

############################# using get_dummies to fill out discrete features ################
df_6 = pd.get_dummies(df_5)
##############################################################################################

In [8]:
import fancyimpute

############################# using fancy_impute to impute data entries ######################
X = df_6.as_matrix()
mice = fancyimpute.MICE(verbose=0)
X_fancy_mice = mice.complete(X)
#############################################################################################

# Split
X_train, X_val, y_train, y_val = train_test_split(
    X_fancy_mice, y, random_state=0)



Using TensorFlow backend.
/Users/apple/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [9]:
# Evaluate
ridge_pipe = make_pipeline(StandardScaler(), RidgeCV())
scores = cross_val_score(ridge_pipe, X_train, y_train, cv=10)
np.mean(scores), np.std(scores)

(0.95779890941944656, 0.014270645620454577)